In [6]:
import os
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import vertexai
from vertexai.generative_models import GenerativeModel

# SCOPES: Permissions we are requesting
SCOPES = [
    'https://www.googleapis.com/auth/documents',    # Docs
    'https://www.googleapis.com/auth/forms.body',   # Forms
    'https://www.googleapis.com/auth/drive.file',   # Drive (Created files only)
    'https://www.googleapis.com/auth/gmail.compose' # Gmail (Drafts only)
]

def authenticate_user():
    """Handles the user login flow (OAuth 2.0)."""
    creds = None
    # Check if we already have a valid token from a previous login
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        
    # If no valid token, let the user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists('credentials.json'):
                print("❌ ERROR: credentials.json not found. Please move it to this folder.")
                return None
            
            # Launch the browser for login
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            
        # Save the token for next time
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            
    return creds

In [7]:
def init_orchestrator():
    # Initialize Vertex AI
    # Replace with your actual Project ID from Google Cloud
    PROJECT_ID = "edu-teacher-assistant-prod" 
    vertexai.init(project=PROJECT_ID, location="us-central1")
    
    model = GenerativeModel("gemini-2.0-flash-exp")
    print("🧠 Orchestrator (Gemini 2.0) initialized.")
    return model

In [8]:
def main():
    print("🚀 Starting AI Teacher Assistant...")
    
    # 1. Authenticate Workspace
    creds = authenticate_user()
    if not creds:
        return
    print("✅ Workspace Authentication successful.")

    # 2. Initialize AI
    try:
        model = init_orchestrator()
        # 3. Simple Test
        response = model.generate_content("Hello! Please confirm you are ready to assist.")
        print(f"\n🤖 Agent Response: {response.text}")
    except Exception as e:
        print(f"\n❌ AI Error: {e}")
        print("Tip: Ensure you have enabled the 'Vertex AI API' in Google Cloud Console.")

In [9]:
main()

🚀 Starting AI Teacher Assistant...
✅ Workspace Authentication successful.
🧠 Orchestrator (Gemini 2.0) initialized.


/home/timmy/ai-teacher-assistant-v2-ai/.venv/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()



❌ AI Error: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.
Tip: Ensure you have enabled the 'Vertex AI API' in Google Cloud Console.


### Agent test

In [1]:
from google.genai import types
from google.adk.agents import Agent
from vertexai.agent_engines import AdkApp

In [2]:
model = "gemini-2.0-flash"

In [4]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

generate_content_config = types.GenerateContentConfig(
    safety_settings=safety_settings,
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.8,
)

agent = Agent(
    model=model,
    name = "currency_exchange_agent",
    generate_content_config=generate_content_config,
)

In [10]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "vertex-key.json"
PROJECT_ID = "edu-teacher-assistant-prod" 
vertexai.init(project=PROJECT_ID, location="us-central1")

In [12]:
app = AdkApp(
    agent=agent
)

async for event in app.async_stream_query(
   user_id="user-123",  # Required
   message="What is the exchange rate from US dollars to Swedish currency?",
):
   print(event)

App name mismatch detected. The runner is configured with app name "default-app-name", but the root agent was loaded from "/home/timmy/ai-teacher-assistant-v2-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "default-app-name", but the root agent was loaded from "/home/timmy/ai-teacher-assistant-v2-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


{'model_version': 'gemini-2.0-flash', 'content': {'parts': [{'text': "Please specify which Swedish currency you are referring to. Sweden's currency is the Swedish Krona (SEK). Once you confirm that is what you are looking for, I can provide the exchange rate.\n"}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 42, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 42}], 'prompt_token_count': 28, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 28}], 'total_token_count': 70, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.21276451292492093, 'invocation_id': 'e-cde5f76a-b079-4905-839b-d3128465aac6', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': '46248e32-dcbd-420d-bba5-f2a281d68eb9', 'timestamp': 1764922161.384171}
